In [1]:
import warnings
import numpy as np
import pandas as pd
import copy
from pathlib import Path
import torch
import torch.nn as nn

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

import pytorch_forecasting
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters
from lightning.pytorch.tuner import Tuner

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\pytorch_forecasting\models\base\_base_model.py:28: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
df_train = pd.read_csv('D:/4. Seminar/F. Grease_Bearing RUL detection/1. Bearing diagnosis (Grease fault and others)/SpindleVibration/predict-future-sales/sales_train.csv' ,parse_dates=['date'])
df_shops = pd.read_csv('D:/4. Seminar/F. Grease_Bearing RUL detection/1. Bearing diagnosis (Grease fault and others)/SpindleVibration/predict-future-sales/shops.csv')
df_items = pd.read_csv('D:/4. Seminar/F. Grease_Bearing RUL detection/1. Bearing diagnosis (Grease fault and others)/SpindleVibration/predict-future-sales/items.csv')
df_item_categories = pd.read_csv('D:/4. Seminar/F. Grease_Bearing RUL detection/1. Bearing diagnosis (Grease fault and others)/SpindleVibration/predict-future-sales/item_categories.csv')

In [3]:
df_train['date'] = pd.to_datetime(df_train['date'], errors='coerce')
print(df_train.columns)
# change the item count per day to item count per month by using group
df_train = df_train.groupby(["item_id","shop_id","date_block_num"]).sum(numeric_only=True).reset_index()
df_train = df_train.rename(index=str, columns = {"item_cnt_day":"item_cnt_month"})
df_train = df_train[["item_id","shop_id","date_block_num","item_cnt_month"]]
print(df_train.columns)
df_train.head(5)

Index(['date', 'date_block_num', 'shop_id', 'item_id', 'item_price',
       'item_cnt_day'],
      dtype='object')
Index(['item_id', 'shop_id', 'date_block_num', 'item_cnt_month'], dtype='object')


,item_id,shop_id,date_block_num,item_cnt_month
0,0,54,20,1.0
1,1,55,15,2.0
2,1,55,18,1.0
3,1,55,19,1.0
4,1,55,20,1.0


In [4]:
df_train['date_block_num'].describe()

count    1.609124e+06
mean     1.466479e+01
std      9.542322e+00
min      0.000000e+00
25%      6.000000e+00
50%      1.400000e+01
75%      2.300000e+01
max      3.300000e+01
Name: date_block_num, dtype: float64

In [5]:
df_train['date_block_num'].max()

np.int64(33)

In [6]:
max_prediction_length = 1
max_encoder_length = 27
training_cutoff = df_train['date_block_num'].max() - max_prediction_length
print(training_cutoff)

32


In [7]:
training = TimeSeriesDataSet(
    df_train[lambda x: x['date_block_num'] <= training_cutoff],
    time_idx='date_block_num',
    target="item_cnt_month",
    group_ids=["shop_id", "item_id"],
    min_encoder_length=0,  
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=[],
    static_reals=["shop_id", "item_id"],
    time_varying_known_categoricals=[],  
    time_varying_known_reals=['date_block_num'],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=['date_block_num'],
    categorical_encoders={'shop_id': pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True),
                          'item_id':pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True)},
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True)

In [8]:
validation = TimeSeriesDataSet.from_dataset(training, df_train, predict=True, stop_randomization=True)

batch_size = 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

In [13]:
pl.seed_everything(42)
trainer = pl.Trainer(
    devices=1,
    # clipping gradients is a hyperparameter and important to prevent divergence
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    # not meaningful for finding the learning rate but otherwise very important
    learning_rate=0.03,
    hidden_size=16,  # most important hyperparameter apart from learning rate
    # number of attention heads. Set to up to 4 for large datasets
    attention_head_size=1,
    dropout=0.1,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=8,  # set to <= hidden_size
    output_size=1,  # 7 quantiles by default
    loss=pytorch_forecasting.metrics.RMSE(),
    # reduce learning rate if no improvement in validation loss after x epochs
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

Seed set to 42
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Number of parameters in network: 19.4k


In [16]:
# find optimal learning rate
res = Tuner.lr_find(model = tft(),train_dataloaders=train_dataloader,val_dataloaders=val_dataloader,max_lr=0.1,min_lr=1e-7)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()

TypeError: TemporalFusionTransformer.forward() missing 1 required positional argument: 'x'